In [1]:
using WAV
using ACME

fs = 44100

#  DIODE CLIPPER CIRCUIT MODEL
function diodeclipper(::Type{Circuit})
     @circuit begin
         j_in = voltagesource(), [-] ⟷ gnd
         r1 = resistor(1e3), [1] ⟷ j_in[+]
         c1 = capacitor(47e-9), [1] ⟷ r1[2], [2] ⟷ gnd
         d1 = diode(is=1e-15), [-] ⟷ gnd, [+] ⟷ r1[2]
         d2 = diode(is=1.8e-15), [-] ⟷ r1[2], [+] ⟷ gnd
         j_out = voltageprobe(), [-] ⟷ gnd, [+] ⟷ r1[2]
     end
 end
 
 diodeclipper(::Type{DiscreteModel}=DiscreteModel; fs=44100, solver=nothing) =
     solver === nothing ?
         DiscreteModel(diodeclipper(Circuit), 1//fs) :
         DiscreteModel(diodeclipper(Circuit), 1//fs, solver)

diodeclipper (generic function with 3 methods)

In [2]:
#  SALLEN KEY 2ND ORDER LP FILTER MDODEL
function sallenkey(::Type{Circuit})
     @circuit begin
         j_in = voltagesource(), [-] ⟷ gnd
         r1 = resistor(10e3), [1] ⟷ j_in[+]
         r2 = resistor(10e3), [1] ⟷ r1[2]
         c1 = capacitor(10e-9), [1] ⟷ r1[2]
         u1 = opamp(), ["in+"] ⟷ r2[2], ["in-"] ⟷ ["out+"] ⟷ c1[2], ["out-"] ⟷ gnd
         #u1 = opamp(Val{:macak}, 1000, -4, 4)
         c2 = capacitor(10e-9), [1] ⟷ u1["in+"], [2] ⟷ gnd
         j_out = voltageprobe(), [-] ⟷ gnd, [+] ⟷ u1["out+"]
     end
 end
 
 sallenkey(::Type{DiscreteModel}=DiscreteModel; fs=44100, solver=nothing) =
     solver === nothing ?
         DiscreteModel(sallenkey(Circuit), 1//fs) :
         DiscreteModel(sallenkey(Circuit), 1//fs, solver)

sallenkey (generic function with 3 methods)

We can calculate the precise cutoff frequency by using the formula 1 / (2 * pi * sqrt(r1 * r2 * c1 * c2))

So for our case this means:

Cutoff Frequency = 1 / (2 * pi * sqrt(10e3 * 10e3 * 10e-9 * 10e-9)) = 1591.55 (hz)

Select either the filter model or the diode model.

In [3]:
diodeModel = diodeclipper(fs=round(Int,fs))
filterModel = sallenkey(fs=round(Int,fs))

# Choose which model you want to use for processing the audio
selectedModel = filterModel 

# Choose an audio file, which you place in the repo directory and replace AudioFile.wav with the correct name.

data_in, fs, _ = wavread("AudioFile.wav")

u = 7 * transpose(data_in[:, 1])
y = run!(selectedModel, u)

data_out = 0.1*transpose(y)
wavplay(data_out, fs)

This section will just print out a processed mono audio file.

In [5]:
if selectedModel == model
    wavwrite(data_out, "Output_diode.wav", Fs=fs)
    data_out, fs = wavread("Output_diode.wav")
    wavappend(data_out, "Output_diode.wav")
elseif selectedModel == model2
    wavwrite(data_out, "Output_filtered.wav", Fs=fs)
    data_out, fs = wavread("Output_filtered.wav")
    wavappend(data_out, "Output_filtered.wav")
end